In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as f

In [2]:
col_names = ['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP']
df_news = pd.read_csv('/Users/ryomisawa/Downloads/NewsAggregatorDataset/newsCorpora.csv', sep = '\t', names=col_names)

df_news = df_news[(df_news['PUBLISHER'] == 'Reuters') | (df_news['PUBLISHER'] ==  'Huffington Post')|(df_news['PUBLISHER'] == 'Businessweek') | (df_news['PUBLISHER'] == 'Contactmusic.com') |(df_news['PUBLISHER'] == 'Daily Mail')].sample(frac=1, random_state=0).reset_index()

df_news['CATEGORY'].replace('b', 0, inplace=True)
df_news['CATEGORY'].replace('t', 1, inplace=True)
df_news['CATEGORY'].replace('e', 2, inplace=True)
df_news['CATEGORY'].replace('m', 3, inplace=True)

In [4]:
df_train, df_valid = train_test_split(df_news, test_size=0.2)
df_valid, df_test = train_test_split(df_valid, test_size=0.5)

In [5]:
df_train.reset_index(inplace=True)
df_valid.reset_index(inplace=True)
df_test.reset_index(inplace=True)
df_train.drop('index', axis=1, inplace=True)
df_valid.drop('index', axis=1, inplace=True)
df_test.drop('index', axis=1, inplace=True)
df_train.drop(436, axis=0, inplace=True)
df_train.reset_index(inplace=True)
df_train.drop('index', axis=1, inplace=True)

In [6]:
title_train = df_train['TITLE']
y_train = df_train['CATEGORY']
#y_train = torch.tensor(y_train).long()
title_valid = df_valid['TITLE']
y_valid = df_valid['CATEGORY']
#y_valid = torch.tensor(y_valid).long()
title_test = df_test['TITLE']
y_test = df_test['CATEGORY']
#y_test = torch.tensor(y_test).long()

In [16]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/ryomisawa/Downloads/GoogleNews-vectors-negative300.bin', binary=True)

In [17]:
d = 300

x_train = []

for text in title_train:
    text = text.split()
    t = len(text)
    x = 0
    for word in text:
        if word not in model:
            x += np.zeros(d)
        else:
            x += model[word]
    x = x/t
    x_train.append(list(x))

In [18]:
x_valid = []

for text in title_valid:
    text = text.split()
    t = len(text)
    x = 0
    for word in text:
        if word not in model:
            x += np.zeros(d)
        else:
            x += model[word]
    x = x/t
    x_valid.append(list(x))
x_valid = torch.tensor(x_valid).float()

In [19]:
x_test = []

for text in title_test:
    text = text.split()
    t = len(text)
    x = 0
    for word in text:
        if word not in model:
            x += np.zeros(d)
        else:
            x += model[word]
    x = x/t
    x_test.append(list(x))
x_test = torch.tensor(x_test).float()

In [20]:
L = 4
W = torch.randn(d, L, requires_grad=True)
x_1 = torch.tensor(x_train[0])
x_1_4 = torch.tensor([x_train[0], x_train[1], x_train[2], x_train[3]])
y_1 = f.softmax(torch.matmul(x_1, W))
Y = f.softmax(torch.matmul(x_1_4, W))

In [21]:
l_1 = -torch.log(y_1[y_train[0]])
l_1_4 = 0
for i in range(4):
    l_1_4 += -torch.log(Y[i][y_train[i]])
l_1_4 = l_1_4/4

In [22]:
l_1.backward()
W.grad

tensor([[ 3.0262e-03,  6.8466e-02, -7.2572e-02,  1.0798e-03],
        [ 1.6893e-03,  3.8219e-02, -4.0511e-02,  6.0278e-04],
        [-6.5253e-04, -1.4763e-02,  1.5648e-02, -2.3284e-04],
        ...,
        [ 4.5177e-04,  1.0221e-02, -1.0834e-02,  1.6121e-04],
        [ 4.5096e-04,  1.0203e-02, -1.0815e-02,  1.6092e-04],
        [-2.7411e-04, -6.2017e-03,  6.5736e-03, -9.7813e-05]])

In [23]:
l_1_4.backward()
W.grad

tensor([[ 2.5744e-02,  8.9736e-02, -1.1825e-01,  2.7675e-03],
        [-6.8984e-03,  5.4763e-02, -5.0717e-02,  2.8516e-03],
        [-1.3204e-03, -2.2393e-02,  2.3014e-02,  6.9902e-04],
        ...,
        [ 1.5911e-03,  7.7844e-03, -5.4977e-03, -3.8779e-03],
        [-3.1871e-02,  1.9434e-02,  5.5613e-03,  6.8757e-03],
        [ 1.3581e-02, -9.1696e-03, -4.2967e-03, -1.1477e-04]])

In [54]:
x_train = torch.tensor(x_train).float()
y_train = torch.tensor(y_train).long()
x_valid = torch.tensor(x_valid).float()
y_valid = torch.tensor(y_valid).long()
x_test = torch.tensor(x_test).float()
y_test = torch.tensor(y_test).long()

In [ ]:
x_train = torch.tensor(x_train).float()
epoch = 101
W = torch.randn(d, L, requires_grad=True)
optimizer = optim.SGD([W], lr=0.1)
for i in range(epoch):
     y = f.softmax(torch.matmul(x_train, W))
     l = 0
     for j in range(len(x_train)):
         l += -torch.log(y[j][y_train[j]])
     l = l/len(x_train)

     optimizer.zero_grad()
     l.backward()

     optimizer.step()
     
     if i % 10 == 0:
         print(l.item())

In [208]:
y_pred_train = f.softmax(torch.matmul(x_train, W))
y_pred_test = f.softmax(torch.matmul(x_test, W))

In [209]:
print('Train Accuracy: {:.3f}, Test Accuracy: {:.3f}'.format(accuracy_score(y_train, y_pred_train.argmax(axis=1)), accuracy_score(y_test, y_pred_test.argmax(axis=1))))


Train Accuracy: 0.390, Test Accuracy: 0.373


In [ ]:
import matplotlib.pyplot as plt
W = torch.randn(d, L, requires_grad=True)
loss_train = []
loss_valid = []
acc_train = []
acc_valid = []
optimizer = optim.SGD([W], lr=0.1)
for i in range(epoch):
     pred_train = f.softmax(torch.matmul(x_train, W))
     l_train = 0
     for j in range(len(x_train)):
         l_train += -torch.log(pred_train[j][y_train[j]])
     l_train = l_train/len(x_train)

     optimizer.zero_grad()
     l_train.backward()

     optimizer.step()

     pred_valid = f.softmax(torch.matmul(x_valid, W))
     l_valid = 0
     for j in range(len(x_valid)):
         l_valid += -torch.log(pred_valid[j][y_valid[j]])
     l_valid = l_valid/len(x_valid)

     y_pred_train = f.softmax(torch.matmul(x_train, W))
     y_pred_valid = f.softmax(torch.matmul(x_valid, W))
     
     loss_train.append(l_train.item())
     loss_valid.append(l_valid.item())
     acc_train.append(accuracy_score(y_train, y_pred_train.argmax(axis=1)))
     acc_valid.append(accuracy_score(y_valid, y_pred_valid.argmax(axis=1)))

     x = [i for i in range(len(loss_train))]
     plt.plot(x, loss_train)
     plt.plot(x, loss_valid)
     plt.plot(x, acc_train)
     plt.plot(x, acc_valid)
     plt.show()
     plt.close()

In [37]:
W = torch.randn(d, L, requires_grad=True)

loss_train = []
loss_valid = []
acc_train = []
acc_valid = []
optimizer = optim.SGD([W], lr=0.1)
for i in range(10):
     pred_train = f.softmax(torch.matmul(x_train, W))
     l_train = 0
     for j in range(len(x_train)):
         l_train += -torch.log(pred_train[j][y_train[j]])
     l_train = l_train/len(x_train)

     optimizer.zero_grad()
     l_train.backward()

     optimizer.step()

     pred_valid = f.softmax(torch.matmul(x_valid, W))
     l_valid = 0
     for j in range(len(x_valid)):
         l_valid += -torch.log(pred_valid[j][y_valid[j]])
     l_valid = l_valid/len(x_valid)

     y_pred_train = f.softmax(torch.matmul(x_train, W))
     y_pred_valid = f.softmax(torch.matmul(x_valid, W))
     
     loss_train.append(l_train.item())
     loss_valid.append(l_valid.item())
     acc_train.append(accuracy_score(y_train, y_pred_train.argmax(axis=1)))
     acc_valid.append(accuracy_score(y_valid, y_pred_valid.argmax(axis=1)))

     #x = [i for i in range(len(loss_train))]
     #plt.plot(x, loss_train)
     #plt.plot(x, loss_valid)
     #plt.plot(x, acc_train)
     #plt.plot(x, acc_valid)
     #plt.show()

     torch.save(
         {
             'epoch':i,
             'weight':W,
             'optimizer_state_dict':optimizer.state_dict()
         },
         'model{}.pth'.format(i)
     )

In [55]:
import time

W = torch.randn(d, L, requires_grad=True)
loss_train = []
loss_valid = []
acc_train = []
acc_valid = []
optimizer = optim.SGD([W], lr=0.1)

num_data_train = len(x_train)
num_data_valid = len(x_valid)
batch_size = 8

for i in range(10):
     t1 = time.time()
     sff_idx_t = np.random.permutation(num_data_train)
     sff_idx_v = np.random.permutation(num_data_valid)

     for idx in range(0, num_data_train, batch_size):
         
         x_batch = x_train[sff_idx_t[idx: idx + batch_size if idx + batch_size < num_data_train else num_data_train]]
         y_batch = y_train[sff_idx_t[idx: idx + batch_size if idx + batch_size < num_data_train else num_data_train]]
         pred_train = f.softmax(torch.matmul(x_batch, W))
         
         l_train = 0
         for i in range(len(y_batch)):
             l_train += -torch.log(pred_train[i][y_batch[i]])
         l_train = l_train/len(x_batch)

         optimizer.zero_grad()
         l_train.backward()

         optimizer.step()
     
     for idx in range(0, num_data_valid, batch_size):
         x_batch = x_valid[sff_idx_v[idx: idx + batch_size if idx + batch_size < num_data_valid else num_data_valid]]
         y_batch = y_valid[sff_idx_v[idx: idx + batch_size if idx + batch_size < num_data_valid else num_data_valid]]
         pred_valid = f.softmax(torch.matmul(x_batch, W))

         l_valid = 0
         for i in range(len(y_batch)):
             l_valid += -torch.log(pred_valid[i][y_batch[i]])
         l_valid = l_valid/len(x_batch)
         y_pred_valid = f.softmax(torch.matmul(x_batch, W))
     t2 = time.time()
     elapsed_time = t2 - t1
     print(f'経過時間:{elapsed_time:.2f}', f'訓練誤差:{l_train.item():.3f}', f'検証誤差:{l_valid.item():.3f}')

     if i == 9:
         times.append(elapsed_time)
     #loss_train.append(l_train.item())
     #loss_valid.append(l_valid.item())
     #acc_train.append(accuracy_score(y_train, y_pred_train.argmax(axis=1)))
     #acc_valid.append(accuracy_score(y_valid, y_pred_valid.argmax(axis=1)))

     #x = [i for i in range(len(loss_train))]
     #plt.plot(x, loss_train)
     #plt.plot(x, loss_valid)
     #plt.plot(x, acc_train)
     #plt.plot(x, acc_valid)
     #plt.show()

     torch.save(
         {
             'epoch':i,
             'weight':W,
             'optimizer_state_dict':optimizer.state_dict()
         },
         'model{}.pth'.format(i)
     )

経過時間:1.20 訓練誤差:1.016 検証誤差:0.834
経過時間:1.36 訓練誤差:1.069 検証誤差:0.883
経過時間:1.35 訓練誤差:1.836 検証誤差:0.353
経過時間:1.27 訓練誤差:0.277 検証誤差:0.493
経過時間:1.10 訓練誤差:0.668 検証誤差:0.145
経過時間:1.24 訓練誤差:1.009 検証誤差:0.296
経過時間:1.23 訓練誤差:0.395 検証誤差:0.950
経過時間:1.34 訓練誤差:0.512 検証誤差:0.257
経過時間:1.21 訓練誤差:0.376 検証誤差:0.098
経過時間:1.26 訓練誤差:0.094 検証誤差:0.213


In [57]:
import time

W = torch.randn(d, L, requires_grad=True)
loss_train = []
loss_valid = []
acc_train = []
acc_valid = []
optimizer = optim.SGD([W], lr=0.1)

num_data_train = len(x_train)
num_data_valid = len(x_valid)
batch_size = 8

device = torch.device('cude' if torch.cuda.is_available() else 'cpu')
for i in range(10):
     t1 = time.time()
     sff_idx_t = np.random.permutation(num_data_train)
     sff_idx_v = np.random.permutation(num_data_valid)

     loss_train = []
     loss_valid = []
     for idx in range(0, num_data_train, batch_size):
         
         x_batch = x_train[sff_idx_t[idx: idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
         y_batch = y_train[sff_idx_t[idx: idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)

         pred_train = f.softmax(torch.matmul(x_batch, W))
         
         l_train = 0
         for i in range(len(y_batch)):
             l_train += -torch.log(pred_train[i][y_batch[i]])
         l_train = l_train/len(x_batch)

         optimizer.zero_grad()
         l_train.backward()

         optimizer.step()

         loss_train.append(l_train.item())
    
     
     for idx in range(0, num_data_valid, batch_size):
         x_batch = x_valid[sff_idx_v[idx: idx + batch_size if idx + batch_size < num_data_valid else num_data_valid]]
         y_batch = y_valid[sff_idx_v[idx: idx + batch_size if idx + batch_size < num_data_valid else num_data_valid]]

         x_batch = x_batch.to(device)
         y_batch = y_batch.to(device)
         
         pred_valid = f.softmax(torch.matmul(x_batch, W))
         

         l_valid = 0
         for i in range(len(y_batch)):
             l_valid += -torch.log(pred_valid[i][y_batch[i]])
         l_valid = l_valid/len(x_batch)
         y_pred_valid = f.softmax(torch.matmul(x_batch, W))

         loss_valid.append(l_valid.item())

     t2 = time.time()
     elapsed_time = t2 - t1
     print(f'経過時間:{elapsed_time}', np.mean(loss_train), np.mean(loss_valid))

     if i == 9:
         times.append(elapsed_time)
     #loss_train.append(l_train.item())
     #loss_valid.append(l_valid.item())
     #acc_train.append(accuracy_score(y_train, y_pred_train.argmax(axis=1)))
     #acc_valid.append(accuracy_score(y_valid, y_pred_valid.argmax(axis=1)))

     #x = [i for i in range(len(loss_train))]
     #plt.plot(x, loss_train)
     #plt.plot(x, loss_valid)
     #plt.plot(x, acc_train)
     #plt.plot(x, acc_valid)
     #plt.show()

     torch.save(
         {
             'epoch':i,
             'weight':W,
             'optimizer_state_dict':optimizer.state_dict()
         },
         'model{}.pth'.format(i)
     )

経過時間:1.1476891040802002 0.9563228543596468 0.698005071211004
経過時間:0.9534740447998047 0.6148715959875838 0.57455320482304
経過時間:0.9485177993774414 0.5243028376580983 0.5193493579051451
経過時間:0.9258489608764648 0.47535558734675576 0.4872455317192449
経過時間:0.92983078956604 0.44382499426329647 0.4659205999916899
経過時間:0.9367928504943848 0.4218704569330816 0.4507430860003133
経過時間:0.9438028335571289 0.4052458920151643 0.4413008286790577
経過時間:0.9247448444366455 0.39218815036656995 0.4321495979176667
経過時間:0.9181609153747559 0.3816246898102014 0.4236053497290718
経過時間:0.9252920150756836 0.3729197781196498 0.41832490665782357
